In [28]:
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.utils.data as Data
import torch

In [29]:
batch_size = 200
learning_rate = 0.01
epochs = 5

train_data = datasets.MNIST('../../../../../Other/datasets/PyTorch',
                            train=True,
                            transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))
                            ]))

test_data = datasets.MNIST('../../../../../Other/datasets/PyTorch',
                           train=False,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ]))

# 手写数字训练数据集
train_loader = Data.DataLoader(dataset=train_data,
                               batch_size=batch_size,
                               shuffle=True)

# 手写数字测试数据集
test_loader = Data.DataLoader(dataset=test_data,
                              batch_size=batch_size,
                              shuffle=True)

In [30]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.featrues = nn.Sequential(  # 内部实现了forward函数;各模块顺序执行
            nn.Conv2d(1, 6, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(400, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10),  # 10分类问题
        )

    def forward(self, x):
        x = self.featrues(x)
        x = x.reshape(x.size()[0], -1)
        x = self.classifier(x)
        return x

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 所有的模型,数据必须都转换到GPU上
criteon = nn.CrossEntropyLoss().to(device=device,
                                   dtype=torch.float32)  # 定义损失函数为交叉熵并转换module的device和dtype(nn.module为in-place)
net = LeNet()
net.to(device=device, dtype=torch.float32)  # 转换module的devic和于dtype(in-place)

# 多参数组的SGD优化器
optimizer = optim.SGD([dict(params=net.featrues.parameters(), momentum=0.9),
                       dict(params=net.classifier.parameters(),
                            momentum=0.95,  # 动量参数\alpha;默认momentum=0
                            nesterov=True)],  # 是否启用Nesterov动量
                      lr=learning_rate)  # 学习率

In [33]:
transforms_data = []
for i, j in train_data:
    transforms_data.append(i.tolist())
transforms_data_ten = torch.tensor(transforms_data)
transforms_data_ten.shape

torch.Size([60000, 1, 28, 28])

In [34]:
transforms_data_loader = Data.DataLoader(transforms_data_ten, batch_size=200, shuffle=True)
y_label = train_loader.dataset.targets

In [35]:
from torch_model.train_evaluate import Train_Evaluate

t_and_v = Train_Evaluate(model=net, optimizer=optimizer, criterion=criteon, epochs=2)

In [36]:
metric_reslut = t_and_v.train_eval(train_loader=train_loader, valid_sets=[(transforms_data_loader, y_label)])
metric_reslut

Train Epoch: 0  [0    /60000 (0  %)]	Loss: 2.300406
Train Epoch: 0  [2000 /60000 (3  %)]	Loss: 2.291150
Train Epoch: 0  [4000 /60000 (7  %)]	Loss: 2.271410
Train Epoch: 0  [6000 /60000 (10 %)]	Loss: 2.245275
Train Epoch: 0  [8000 /60000 (13 %)]	Loss: 2.143907
Train Epoch: 0  [10000/60000 (17 %)]	Loss: 1.713377
Train Epoch: 0  [12000/60000 (20 %)]	Loss: 0.927804
Train Epoch: 0  [14000/60000 (23 %)]	Loss: 1.283235
Train Epoch: 0  [16000/60000 (27 %)]	Loss: 0.703749
Train Epoch: 0  [18000/60000 (30 %)]	Loss: 0.462573
Train Epoch: 0  [20000/60000 (33 %)]	Loss: 0.433865
Train Epoch: 0  [22000/60000 (37 %)]	Loss: 0.367559
Train Epoch: 0  [24000/60000 (40 %)]	Loss: 0.330594
Train Epoch: 0  [26000/60000 (43 %)]	Loss: 0.342789
Train Epoch: 0  [28000/60000 (47 %)]	Loss: 0.234067
Train Epoch: 0  [30000/60000 (50 %)]	Loss: 0.170483
Train Epoch: 0  [32000/60000 (53 %)]	Loss: 0.313330
Train Epoch: 0  [34000/60000 (57 %)]	Loss: 0.245354
Train Epoch: 0  [36000/60000 (60 %)]	Loss: 0.198010
Train Epoch:

[[10.471982955932617], [11.857095718383789]]